In [1]:
import pandas as pd
import os
import glob

# 1. Load Files Here: (extension name needed)
input_folder = 'ESI+_1_sus_screening_0p2ppm/raw/'
output_folder = 'ESI+_9_NIST_0p2ppm/'
csv_files = glob.glob(os.path.join(input_folder, '*.csv'))

# Loop through each file in the folder
for input_file_path in csv_files:
    # Raw Data:
    input_file = pd.read_csv(input_file_path, skiprows=6).dropna()
    # Library File:
    library_file = pd.read_csv('../PFAS_libraries/NIST_PFAS_Suspect_List_pos.csv')
    # Output File:
    output_file_name = os.path.splitext(os.path.basename(input_file_path))[0] + '_known.csv'
    output_file = os.path.join(output_folder, output_file_name)

    # 2. Set Parameters Here:
    S_to_N_threshold = 3
    ppm_error = 0.2
    PROTON = 1.00727647
    SODIUM = 22.989769
    POTASSIUM = 38.963706
    
    # 3. Input File Peaks List
    ## 3a. Check input file
    # input_file = input_file.loc[input_file['S/N'] >= S_to_N_threshold, :]
    raw_pks = len(input_file.index)
    
    ## 3b. Convert all of the uppercase column labels to lowercase and replace the spaces with `_`.
    input_file.columns = (
        input_file.columns.str.lower()
        .str.strip()
        .str.replace(r'\s+', '_', regex=True)
    )
    input_file['nominal_mass'] = round(input_file['peak_location']).astype(int)
    
    ## 3c. Calculate experimental KMD values of CF2, CH2, and C2H4O series
    input_file['KMD_CF2'] = (input_file['peak_location'].round() - input_file['peak_location']*50/49.99681).round(4)
    input_file['KMD_CH2'] = (input_file['peak_location'].round() - input_file['peak_location']*14/14.01565).round(4)
    input_file['KMD_C2H4O'] = (input_file['peak_location'].round() - input_file['peak_location']*44/44.02621).round(4)
    
    # 4. Library Peak List
    ## 4a. Check library file
    lib_pks = len(library_file.index)
    library_file.shape
    # Convert all of the uppercase column labels to lowercase and replace the spaces with `_`.
    library_file.columns = (
        library_file.columns.str.lower()
        .str.strip()
        .str.replace(r'\s+', '_', regex=True)
    )
    
    ## 4b. Compute protonated/deprotonated mass of the library
    list_suspects = library_file.assign(
        mz_pos_H = library_file.loc[:, 'mass'] + PROTON,
        nominal_mz_pos_H = round(library_file.loc[:, 'mass'] + PROTON).astype(int)
    )
    list_suspects.sort_values(by='mass', inplace=True)

    # 5. Suspect List Matches
    def search_suspect_list(input_file, list_suspects, ppm_error):
        matches = pd.DataFrame()
        
        for index, row in input_file.iterrows():
            
            LOWER = row['peak_location'] - row['peak_location'] * ppm_error / 1E6
            UPPER = row['peak_location'] + row['peak_location'] * ppm_error / 1E6
                    
            temp_matches = list_suspects[(list_suspects['mz_pos_H'] >= LOWER) & (list_suspects['mz_pos_H'] <= UPPER)]
            
            if len(temp_matches):
                new_df = temp_matches.copy()
                new_df.loc[:, 'exper_mz'] = row['peak_location']
                new_df.loc[:, 'height'] = row['peak_height']
                new_df.loc[:, 'abund'] = row['scaled_abundance']
                new_df.loc[:, 'exp_KMD_CF2'] = row['KMD_CF2']
                new_df.loc[:, 'exp_KMD_CH2'] = row['KMD_CH2']
                new_df.loc[:, 'exp_KMD_C2H4O'] = row['KMD_C2H4O']
                new_df.loc[:, 'S/N'] = row['s/n']
                new_df.loc[:, 'ppm_error'] = 1E6 * (row['peak_location'] - temp_matches['mz_pos_H']) / temp_matches['mz_pos_H']          
                matches = pd.concat([matches, new_df])
                
        return matches

    results = search_suspect_list(input_file, list_suspects, ppm_error)
    
    # 6. Reformat and Print Results
    results.rename(columns={'chemical_name': 'name'}, inplace=True)
    results.rename(columns={'mass': 'lib_mass'}, inplace=True)
    results.rename(columns={'mz_pos_H': 'library_pos_mz'}, inplace=True)
    results.rename(columns={'c': 'C'}, inplace=True)
    results.rename(columns={'h': 'H'}, inplace=True)
    results.rename(columns={'br': 'Br'}, inplace=True)
    results.rename(columns={'cl': 'Cl'}, inplace=True)
    results.rename(columns={'f': 'F'}, inplace=True)
    results.rename(columns={'i': 'I'}, inplace=True)
    results.rename(columns={'n': 'N'}, inplace=True)
    results.rename(columns={'o': 'O'}, inplace=True)
    results.rename(columns={'p': 'P'}, inplace=True)
    results.rename(columns={'s': 'S'}, inplace=True)
    results.rename(columns={'dbe': 'DBE'}, inplace=True)
    results.columns
    results = results.loc[:, ['name', 'exper_mz', 'height', 'abund', 'lib_mass', 'ppm_error', 'S/N', 'formula', 
                            'C', 'H', 'Br', 'Cl', 'F', 'I', 'N', 'O', 'P', 'S', 'DBE']]
    results = results.reset_index()
    
    # Remove those don't have 'H' (which means they can't be deprotonated)
    # results.drop_duplicates(subset=['formula'],inplace=True)
    results = results.loc[results['formula'].str.contains('H')]
    
    #Count the number of unique peaks have assigned
    num_unique = results.loc[:, 'exper_mz'].nunique()

    # 7. Check the isotopologue
    ## 7a. Define isotopes and calculate iso mass
    C_mass = 12.000000
    Br_mass = 78.918336
    Cl_mass = 34.968853
    O_mass = 15.994915
    S_mass = 31.972072
    # Define the isotopes:
    C_13_mass = 13.003355
    C_13_abund = 0.0107/0.9893
    Br_81_mass = 80.916290	
    Br_81_abund = 0.4931/0.5069
    Cl_37_mass = 36.965903	
    Cl_37_abund = 0.2423/0.7577
    O_18_mass = 17.999159	
    O_18_abund = 0.0020/0.99757
    S_34_mass = 33.967868
    S_34_abund = 0.0421/0.9493
              
    results.loc[results['C'] > 0, '13C_mass'] = results['exper_mz'] + (C_13_mass - C_mass)
    results.loc[results['Br'] > 0, '81Br_mass'] = results['exper_mz'] + (Br_81_mass - Br_mass)
    results.loc[results['Cl'] > 0, '37Cl_mass'] = results['exper_mz'] + (Cl_37_mass - Cl_mass)
    results.loc[results['S'] > 0, '34S_mass'] = results['exper_mz'] + (S_34_mass - S_mass)
    results.loc[results['O'] > 0, '18O_mass'] = results['exper_mz'] + (O_18_mass - O_mass)

    results.loc[results['C'] > 0, '13C_abund'] = results['abund'] * results['C'] * C_13_abund
    results.loc[results['Br'] > 0, '81Br_abund'] = results['abund'] * results['Br'] * Br_81_abund
    results.loc[results['Cl'] > 0, '37Cl_abund'] = results['abund'] * results['Cl'] * Cl_37_abund
    # results.loc[results['Cl'] > 0, '37Cl2_abund'] = results['abund'] * results['Cl'] * (results['Cl'] - 1)/2 * Cl_37_abund * Cl_37_abund
    results.loc[results['S'] > 0, '34S_abund'] = results['abund'] * results['S'] * S_34_abund
    results.loc[results['O'] > 0, '18O_abund'] = results['abund'] * results['O'] * O_18_abund
    
    ## 7b. Search for isotope peaks
    def search_isotope(results, column_name, intensity_error):
        results[column_name + '_iso?'] = ''
        results[column_name + '_abund?'] = ''
        results[column_name + '_abund_err?'] = ''
        results[column_name + '_match?'] = ''
        
        for i, row in results.iterrows():
            ppm_lower_bound = row[column_name + '_mass'] - row[column_name + '_mass'] * ppm_error / 1E6
            ppm_upper_bound = row[column_name + '_mass'] + row[column_name + '_mass'] * ppm_error / 1E6
            intensity_lower_bound = row[column_name + '_abund'] * (1 - intensity_error)
            intensity_upper_bound = row[column_name + '_abund'] * (1 + intensity_error)
            # Check if any 'peaklist' value in raw data CSV falls within the range
            match = ((input_file['peak_location'] >= ppm_lower_bound) & 
                    (input_file['peak_location'] <= ppm_upper_bound) & 
                    (input_file['scaled_abundance'] > intensity_lower_bound) & 
                    (input_file['scaled_abundance'] < intensity_upper_bound))
            
            if any(match):
                # Get the matching 'peak_location' and 'scaled_abundance' values
                matched_peak_location = input_file.loc[match, 'peak_location'].values[0]
                matched_scaled_abundance = input_file.loc[match, 'scaled_abundance'].values[0]
                abundance_error = ((matched_scaled_abundance - row[column_name + '_abund']) / row[column_name + '_abund']).round(4)
                # Assign the matched values to the respective columns
                results.at[i, column_name + '_iso?'] = matched_peak_location
                results.at[i, column_name + '_abund?'] = matched_scaled_abundance
                results.at[i, column_name + '_abund_err?'] = abundance_error
                results.at[i, column_name + '_match?'] = 'Y'
        return results
        
    results = search_isotope(results, '13C', 0.15)
    results = search_isotope(results, '81Br', 0.15)
    results = search_isotope(results, '37Cl', 0.15)
    results = search_isotope(results, '18O', 0.6)
    results = search_isotope(results, '34S', 0.3)
     
    ## 7c. Eliminate those assignments that contain Br/Cl/S/C but the isotopologue does not match. *Keep all if abundance of isotopologue < minimum
    #           those don't have this elem   those isotopologues match          those isopeak's abundance is < minimum
    results = results[(results['Br'] < 1) | (results['81Br_match?'] == 'Y') | ((results['Br'] >= 1) & (results['abund'] < results['abund'].min() / (results['Br'] * Br_81_abund)))]
    results = results[(results['Cl'] < 1) | (results['37Cl_match?'] == 'Y') | ((results['Cl'] >= 1) & (results['abund'] < results['abund'].min() / (results['Cl'] * Cl_37_abund)))]
    results = results[(results['O'] < 1) | (results['18O_match?'] == 'Y') | ((results['O'] >= 1) & (results['abund'] < results['abund'].min() / (results['O'] * O_18_abund)))]
    results = results[(results['S'] < 1) | (results['34S_match?'] == 'Y') | ((results['S'] >= 1) & (results['abund'] < results['abund'].min() / (results['S'] * S_34_abund)))]
    results = results[(results['13C_match?'] == 'Y') | ((results['abund'] < results['abund'].min() / (results['C'] * C_13_abund)))]
    # peak_number = results.groupby('exper_mz').ngroup() + 1
    assgnmnts_number = len(results.index)
    

    results['KMD_CF2'] = (results['lib_mass'].round() - results['lib_mass']*50/49.99681).round(4)
    results['z*_CF2'] = (results['lib_mass'].round() % 50) - 50

    final = results.drop_duplicates(subset=['formula'])
    final = final.loc[:, ['name', 'exper_mz', 'height', 'abund', 'lib_mass', 'ppm_error', 'S/N', 
                            'KMD_CF2', 'z*_CF2', 'formula', 'C', 'H', 'Br', 'Cl', 'F', 'I', 'N', 'O', 'P', 'S', 'DBE', 
                            '13C_match?', '13C_abund_err?', '81Br_match?', '81Br_abund_err?', '37Cl_match?', '37Cl_abund_err?', 
                            '18O_match?', '18O_abund_err?', '34S_match?', '34S_abund_err?']]
    
    final = final.reset_index()
    peak_number = final.groupby('exper_mz').ngroup() + 1
    assgnmnts_number = len(final.index)
    
    # 9. Add counter columns for some stat numbers
    # peak_number_after_KMD = results_unique.groupby('exper_mz').ngroup() + 1
    # final['peak_number'] = peak_number_after_KMD
    # final = final.sort_values('peak_number')
    # results['assignment_number'] = results.groupby('exper_mz').cumcount() + 1     # This counts the # of assignments of a peak
    final.loc[0, '#raw_pks'] = int(raw_pks)
    final.loc[0, '#lib_pks'] = int(lib_pks)
    final.loc[0, '#matched_pks'] = int(peak_number.max())
    final.loc[0, '#assgnmnts'] = int(assgnmnts_number)

    final.to_csv(output_file, index=False)